In [1]:
import random
import numpy as np
import itertools
import copy

### Powerset of S

The function might not be needed for now

In [2]:
def powerset(s):
    x = len(s)
    power_set = []
    for i in range(1 << x):
       power_set.append([s[j] for j in range(x) if (i & (1 << j))])

    return power_set

### F(x) without sampling

In [3]:
def F(x, f, n):
    
    A = [np.array(i) for i in itertools.product([0, 1], repeat = n)]
    
    val_S = 0

    for i in range(len(A)):

        prod_in_s = 1
        prod_not_in_s = 1
        S = []

        for j in range(len(A[i])):
            if A[i][j] == 1:
                prod_in_s = prod_in_s*x[j]
                S.append(j+1)
            else:
                prod_not_in_s = prod_not_in_s*(1-x[j])

        val_S = val_S + f(S)*prod_in_s*prod_not_in_s
    
    return val_S

### Approximation of F(x) using sampling 
With t being the number of samples for estimation

In [4]:
def F_approx(x, f, n):
    
    t= 320
    sum_R = 0
    
    for i in range(t):
            
        x_bar = np.random.uniform(0,1, n)
        r_t = x >= x_bar
        R_t = []
            
        for i in range(len(r_t)):

            if r_t[i] == True:
                R_t.append(i+1)
                
        sum_R = sum_R + f(R_t)
#         print(sum_R)

    return sum_R/t

### Compute First Order Gradients of F

In [5]:
def get_gradient_F_for_i(F, x, f, n, i):


    x_without_i = copy.deepcopy(x)
    x_without_i[i] = 0.0

    x_with_i = copy.deepcopy(x)
    x_with_i[i] = 1.0

    # print('x with xi: ', x_with_i)
    # print('x without xi: ', x_without_i)

    df_dxi = F(x_with_i, f, n) - F(x_without_i, f, n)
    return df_dxi

In [6]:
def get_gradient_F(F, x, f):
    
    n = len(x)
    
    grad = np.zeros(len(x))
    
    for i in range(len(x)):
        
        grad[i] = get_gradient_F_for_i(F, x, f, n, i)
    
    return grad

### Gradient Ascent Method

In [7]:
# stepsize for gradient ascent
alpha = 0.01

def gradient_ascent(F, x, f, n, alpha):
    
    x_init = copy.deepcopy(x)
    sum_init = F(x, f, n)
    # key values to be used
    sum_update = 0
    iter = 0
    sum_temp = copy.deepcopy(sum_init)

    # start updating the parameters x with iterative gradients
    while np.abs(sum_temp - sum_update) > 10 ** (-2):
        iter += 1
        sum_temp = F(x, f, n)

        for i in range(n):
            grad_i = get_gradient_F_for_i(F, x, f, n, i)
            x[i] = np.maximum(np.minimum(x[i] + alpha * grad_i, 1.0), 0.0)

        sum_update = F(x, f, n)

#         print('x updated: ', x)
#         print('sum updated: ', sum_update)

    print('Iterations: ', iter, '\n')
    print('Initial F: ', sum_init)
    print('Initial x: ', x_init, '\n')
    print('Final F: ', sum_update)
    print('Final x: ', x, '\n')
    return iter, sum_update, x

## Testing

With different versions of f: 2^S -> R

Define the size of the problem (n) 

In [8]:
n = 10
N = [(i+1) for i in range(n)]

Initialize an x vector

In [9]:
x = np.random.uniform(0,1, n)
print("x: ", x)

x:  [0.60864154 0.59242361 0.28103739 0.0399254  0.28637369 0.20680814
 0.06964326 0.2896193  0.09642411 0.3582047 ]


The corresponding S vector with 0.5 as the cutting line

In [10]:
x_bar = 0.5*np.ones(len(x))
s = x >= x_bar
S = []
            
for i in range(len(s)):
    if s[i] == True:
        S.append(i+1)

print("S: ", S)

S:  [1, 2]


## Constant function f(s)

In [11]:
def f_constant(S):
    return 5

### f_constant(S): Values of the function 

Actual value

In [12]:
f_constant(S)

5

Multi-linear extenstion

In [13]:
F(x, f_constant, n)

4.999999999999996

Mutli-linear extension with sampling

In [14]:
F_approx(x, f_constant, n)

5.0

### f_constant(S): Values of the gradients using multi-linear extension

without sampling

In [15]:
get_gradient_F(F_approx, x, f_constant)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

With sampling

In [16]:
get_gradient_F(F, x, f_constant)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
gradient_ascent(F_approx, x, f_constant, n, alpha)

Iterations:  1 

Initial F:  5.0
Initial x:  [0.60864154 0.59242361 0.28103739 0.0399254  0.28637369 0.20680814
 0.06964326 0.2896193  0.09642411 0.3582047 ] 

Final F:  5.0
Final x:  [0.60864154 0.59242361 0.28103739 0.0399254  0.28637369 0.20680814
 0.06964326 0.2896193  0.09642411 0.3582047 ] 



(1,
 5.0,
 array([0.60864154, 0.59242361, 0.28103739, 0.0399254 , 0.28637369,
        0.20680814, 0.06964326, 0.2896193 , 0.09642411, 0.3582047 ]))

## Linear function f(S)

In [18]:
a = np.random.uniform(-1,1, n)
a

array([-0.59340562, -0.46974944,  0.78174473, -0.15895207, -0.31314228,
        0.36204542,  0.75484021, -0.38326327, -0.96412591,  0.97059805])

In [19]:
def f_linear(S):
    
    #convert S to 0,1
    s_hat = np.zeros(n)
    
    for i in range(len(S)):
        
        s_hat[S[i]-1] = 1
        
    return np.dot(a,s_hat)

### f_linear: Values of the function

Actual value

In [20]:
f_linear(S)

-1.063155056177732

Multi-linear extenstion

In [21]:
F(x, f_linear, n)

-0.2446335748928383

Mutli-linear extension with sampling

In [22]:
F_approx(x, f_linear, n)

-0.24874924686329497

### f_linear: Values of the gradient using multi-linear extension

without sampling

In [23]:
get_gradient_F(F_approx, x, f_linear)

array([-0.71868911, -0.49783078,  0.82600399, -0.08793915, -0.31921905,
        0.34367244,  0.78437371, -0.27998569, -0.94639274,  0.92237508])

With sampling

In [24]:
get_gradient_F(F, x, f_linear)

array([-0.59340562, -0.46974944,  0.78174473, -0.15895207, -0.31314228,
        0.36204542,  0.75484021, -0.38326327, -0.96412591,  0.97059805])

In [25]:
gradient_ascent(F_approx, x, f_linear, n, alpha)

Iterations:  5 

Initial F:  -0.15323869856413377
Initial x:  [0.60864154 0.59242361 0.28103739 0.0399254  0.28637369 0.20680814
 0.06964326 0.2896193  0.09642411 0.3582047 ] 

Final F:  -0.0635314119519878
Final x:  [0.57851936 0.56959181 0.31921225 0.0300579  0.27056291 0.22581036
 0.10726583 0.27336616 0.04600029 0.40545071] 



(5,
 -0.0635314119519878,
 array([0.57851936, 0.56959181, 0.31921225, 0.0300579 , 0.27056291,
        0.22581036, 0.10726583, 0.27336616, 0.04600029, 0.40545071]))

## Polynomial f(S) function

In [26]:
def f_polynomial(S):
    
    #convert S to 0,1
    s_hat = np.zeros(n)
    
    for i in range(len(S)):
        
        s_hat[S[i]-1] = 1
        
    a_hat = s_hat*a
    
    return np.dot(a_hat,a_hat)

### f_polynomial: Values of function

Actual value

In [27]:
f_polynomial(S)

0.5727947625533779

Multi-linear extension 

In [28]:
F(x, f_polynomial, n)

1.1073616955006014

Multi-linear extenstion with sampling

In [29]:
F_approx(x, f_polynomial, n)

1.1013232195713976

### f_polynomial: Values of gradients using multi-linear approximation

Without sampling

In [30]:
get_gradient_F(F_approx, x, f_polynomial)

array([0.27245502, 0.20252281, 0.55593714, 0.03396413, 0.06243236,
       0.02377324, 0.5370976 , 0.2007551 , 0.91667946, 0.9513077 ])

With sampling

In [31]:
get_gradient_F(F, x, f_polynomial)

array([0.35213023, 0.22066453, 0.61112482, 0.02526576, 0.09805809,
       0.13107689, 0.56978374, 0.14689073, 0.92953876, 0.94206058])

In [32]:
gradient_ascent(F, x, f_polynomial, n, alpha)

Iterations:  104 

Initial F:  1.1073616955006014
Initial x:  [0.57851936 0.56959181 0.31921225 0.0300579  0.27056291 0.22581036
 0.10726583 0.27336616 0.04600029 0.40545071] 

Final F:  3.5108633219924843
Final x:  [0.9447348  0.79908292 0.95478206 0.05633429 0.37254332 0.36213033
 0.69984092 0.42613252 1.         1.        ] 



(104,
 3.5108633219924843,
 array([0.9447348 , 0.79908292, 0.95478206, 0.05633429, 0.37254332,
        0.36213033, 0.69984092, 0.42613252, 1.        , 1.        ]))